In [19]:
import json
with open ('bronze4_matches_all.json', 'r',  encoding="utf-8") as f:
    all_matches=json.load(f)

In [20]:
len (all_matches)

1000

In [21]:
all_matches[0].keys()

dict_keys(['metadata', 'info'])

In [22]:
all_matches[0]['metadata'].keys()

dict_keys(['dataVersion', 'matchId', 'participants'])

In [23]:
all_matches[0]['info'].keys()

dict_keys(['endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [24]:
solo_ranked_matches = [m for m in all_matches if m["info"]["queueId"] == 420]

len(solo_ranked_matches)


1000

In [25]:
clean_matches=[]

for m in all_matches:
    gameduration=m["info"]['gameDuration']
    if gameduration<=480:
        continue
    clean_matches.append(m)
    
len(clean_matches)
    

967

In [26]:
def is_afk_like(p, game_duration):
    time_played = p.get("timePlayed", game_duration)  
    minutes = game_duration / 60

    
    if time_played < 0.6 * game_duration:
        return True

  
    if minutes >= 10:
        cs = p.get("totalMinionsKilled", 0)
        cs_per_min = cs / minutes
        k = p.get("kills", 0)
        a = p.get("assists", 0)
        dmg = p.get("totalDamageDealtToChampions", 0)
        gold = p.get("goldEarned", 0)

        if cs_per_min < 0.5 and (k + a) <= 1 and dmg < 4000 and gold < 4000:
            return True

    return False


In [27]:
final_matches = []

for m in clean_matches:   
    gdur = m["info"]["gameDuration"]
    participants = m["info"]["participants"]
    if any(is_afk_like(p, gdur) for p in participants):
        continue  
    
    final_matches.append(m)

len(final_matches)


962

In [28]:
def is_smurf_like(p, game_duration):
    
    minutes = game_duration / 60.0
    


   
    cs = p.get("totalMinionsKilled", 0) + p.get("neutralMinionsKilled", 0)
    cs_per_min = cs / minutes


    if cs_per_min >= 9.0:
        return True

   
    kills = p.get("kills", 0)
    deaths = p.get("deaths", 0)

    
    if kills > 20 and deaths <= 2:
        return True

    return False

In [29]:
final_clean_matches=[]
for m in clean_matches:
    
    info = m["info"]
    gdur = info["gameDuration"]
    participants = info["participants"]
    

    
    if any(is_smurf_like(p, gdur) for p in participants):
        continue 
    
    
    final_clean_matches.append(m)

In [30]:
len(final_clean_matches)

904

In [31]:
print (final_clean_matches[0]['info']['participants'][0]['puuid'])
print(final_clean_matches[0]['info']['participants'][0]['championId'])

LdJAqo-Gf4iLb-M5-RsVrReOj3dnZuhk_zbujCOVViCC0GtFhDFSFh1ZeqIajJRG0L1Vjv3oLewLrA
516


In [32]:
import os


API_KEY_FILE = "riot_secret.txt" 

try:

    with open(API_KEY_FILE, 'r') as f:
        API_KEY = f.read().strip()
        
        
except FileNotFoundError:
    
    API_KEY = None 
    
except Exception as e:
   
    API_KEY = None


HEADERS = {"X-Riot-Token": API_KEY}

In [33]:
import requests
import time

HEADERS = {"X-Riot-Token": API_KEY}

def get_mastery(puuid, champion_id):
   
    url = f"https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}/by-champion/{champion_id}"
    
    response = requests.get(url, headers=HEADERS)
    
    
    
    if response.status_code == 200:
    
        data = response.json()
        return data["championPoints"]
        
    elif response.status_code == 404:
        
        return 0 
        
    elif response.status_code == 429:
        
        
        time.sleep(10)
        
        return get_mastery(puuid, champion_id)
        
    else:
        
        print(f"Hata kodu: {response.status_code}")
        return None

In [34]:
get_mastery('xFHUllnq3N6GwZ-NoJBYfPRW6XmZxLyBTrBwbkYDZsUTQ8je_5Z3DYYZ_treE4-jxMW5SvbTMs8wuQ', 98)

353530

In [35]:
import pandas as pd
import time

dataset = []

total_matches = len(final_clean_matches)

for match_index, m in enumerate(final_clean_matches, start=1):
    match_id = m["metadata"]["matchId"]
    participants = m["info"]["participants"]

    print(f"\n📌 İşleniyor: Match {match_index}/{total_matches} → MatchID: {match_id}")

    for participant_index, p in enumerate(participants, start=1):
        print(f"   → Participant {participant_index}/10: {p['championName']} ({p['teamPosition']})", end=" ")

        points = get_mastery(p["puuid"], p["championId"])

        if points is None:
            print("❌ (Mastery ALINAMADI)")
            continue
        else:
            print(f"✔ Mastery: {points}")

        row = {
            "matchId": match_id,
            "teamId": p["teamId"],
            "win": 1 if p["win"] else 0,
            "role": p["teamPosition"],
            "championName": p["championName"],
            "masteryPoints": points
        }
        dataset.append(row)

        print(f"      → Dataset boyutu: {len(dataset)} satır")

        time.sleep(1.2)


df = pd.DataFrame(dataset)
df.to_csv("mastery_verisi.csv", index=False)

print("\n\n🎉✔ İŞLEM TAMAMLANDI!")
print(df.head())



📌 İşleniyor: Match 1/904 → MatchID: NA1_5426338970
✔ Mastery: 29432 1/10: Ornn (TOP) 
      → Dataset boyutu: 1 satır
   → Participant 2/10: Udyr (JUNGLE) ✔ Mastery: 540227
      → Dataset boyutu: 2 satır
   → Participant 3/10: Ahri (MIDDLE) ✔ Mastery: 42688
      → Dataset boyutu: 3 satır
   → Participant 4/10: Caitlyn (BOTTOM) ✔ Mastery: 26630
      → Dataset boyutu: 4 satır
   → Participant 5/10: Nautilus (UTILITY) ✔ Mastery: 275659
      → Dataset boyutu: 5 satır
   → Participant 6/10: Teemo (TOP) ✔ Mastery: 1641771
      → Dataset boyutu: 6 satır
   → Participant 7/10: Kayn (JUNGLE) ✔ Mastery: 73146
      → Dataset boyutu: 7 satır
   → Participant 8/10: Gnar (MIDDLE) ✔ Mastery: 79881
      → Dataset boyutu: 8 satır
   → Participant 9/10: MissFortune (BOTTOM) ✔ Mastery: 972711
      → Dataset boyutu: 9 satır
   → Participant 10/10: Seraphine (UTILITY) ✔ Mastery: 68481
      → Dataset boyutu: 10 satır

📌 İşleniyor: Match 2/904 → MatchID: NA1_5285765566
✔ Mastery: 42568 1/10: Sett (

In [36]:
df.head()

,matchId,teamId,win,role,championName,masteryPoints
0,NA1_5426338970,100,1,TOP,Ornn,29432
1,NA1_5426338970,100,1,JUNGLE,Udyr,540227
2,NA1_5426338970,100,1,MIDDLE,Ahri,42688
3,NA1_5426338970,100,1,BOTTOM,Caitlyn,26630
4,NA1_5426338970,100,1,UTILITY,Nautilus,275659


In [39]:
df.to_json("mastery_verisi.json", orient="records", indent=4)
